In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
#req packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import seaborn as sns
import glob
from tqdm import tqdm
from sklearn.preprocessing import minmax_scale

#edf
import mne
#myfuncs
from ecog_functions import *

In [2]:
#function to center and filter the data
def filter_channel(channel, fstart=0.1, fstop=45, sr=1000, center=True, notch_50=False):
    """A function to perform notch filtering, mean centering and bandpass filtering for a single recorded channel. 
    
    Args:
        channel (pandas series/column): A single column from the recording.
        fstart (float, optional): The low end of the range to return. Defaults to 0.1.
        fstop (int, optional): The high end of the range to return. Defaults to 45.
        sr (int, optional): Sampling rate of the recording. Defaults to 1000.
        center (bool, optional): Should centering be performed. Defaults to True.
        notch_50 (bool, optional): Should a 50 Hz Notch filter be applied. Defaults to False, because by default 50 Hz is already out of range.

    Returns:
        Output of the filtering function: a filtered channel.
    """
    if center==True:
        channel=channel-channel.mean()
        
    if notch_50==True:
        b0, a0=signal.iirnotch(50, Q=30, fs=sr)
        channel=signal.filtfilt(b0, a0, channel)
    
    sos_bandpass=signal.butter(N=20, Wn=(fstart, fstop), btype='bp', output='sos',  fs=sr)
    
    return signal.sosfiltfilt(sos_bandpass, channel) 

def sort_cls_by_criteria(grouped):
    """Sorts clusters by a decision tree. 
    Splits by EMG, then by alpha*delta, then beta*gamma, then theta - needs an update
    Modify this function to modify results

    Args:
        grouped (dataframe): dataframe of clusters and their median parameters

    Returns:
        w_cls: clusters marked as wake
        same for other outputs
    """
        
    w_cls = []
    nrem_cls = []
    rem_cls = []
    
    unsorted_cls = grouped.index
    for cluster in unsorted_cls:
        #option 5 - HIGH or not, no ratios
        nonzero = grouped.iloc[:,0:6]>grouped.iloc[:,0:6].quantile(0.25)
        high = grouped.iloc[:,0:6]>grouped.iloc[:,0:6].quantile(0.5)
        very_high = grouped.iloc[:,0:6]>grouped.iloc[:,0:6].quantile(0.75)
        #NREM
        #if not high['emg'][cluster]:
        if high['delta'][cluster] or high['alpha'][cluster] or high['theta'][cluster]:
            if very_high['theta'][cluster] and (nonzero['beta'][cluster] and nonzero['gamma'][cluster]) and not nonzero['emg'][cluster]:
                rem_cls.append(cluster)
            elif (high['delta'][cluster] or high['alpha'][cluster] or high['theta'][cluster]) and not high['emg'][cluster]:
                nrem_cls.append(cluster)
            else:
                w_cls.append(cluster)
        else:
            w_cls.append(cluster)
    #make sure there is at least one REM cluster detected
    #if there are no REM clusters, select the one with highest theta, lowest emg and nonzero beta/gamma
    if not rem_cls:
        #scale gr to allow index calculation
        grouped.iloc[:,:-2] = minmax_scale(grouped.iloc[:,:-2])
        #rem index
        rem_ind = (grouped['theta']**2 * grouped['beta']*grouped['gamma'])/(grouped['emg']**2)
        rem_cls.append(np.where(rem_ind == rem_ind.max())[0][0])
        
    return w_cls, nrem_cls, rem_cls

def integrate_by_mode(data, integration_s = 5):
    """Integrates data by the mode of selected interval - replaces all values with the most common. 
    Works as a rolling filter.

    Args:
        data (pandas series): data of sleep detections (in this case) - N, R, and W categories. 
        integration_s (int, optional): size of window. Defaults to 5.

    Returns:
        res_sleep: list of updated results.
    """
    #temporal integration - select most common state in set s to drop a bit of noise
    res_sleep=[]
    #nans bfilled - should figure out a better method, ran out of time
    raw_sleep=data.reset_index(drop=True).fillna(method='bfill')
    for i in tqdm(range(0, len(raw_sleep)-integration_s)):
        #take most common value in 5s if it exists (not NaN)
        mode = raw_sleep[i:i+integration_s].mode()[0]
        #print(mode)
        if mode !=0:
            res_sleep.append(mode[0])
        else:
            print('weird error, mode=0')
    return res_sleep

In [3]:
#read paths of data published in https://zenodo.org/records/5227351
paths = glob.glob(r'C:\Users\marty\Downloads\5227351\*.edf')
#manual scores path
mansc_path = r"C:\Users\marty\Downloads\5227351\manual_scoring_all_rats.csv"

In [4]:
#glob params
SAMPLE_RATE = 250

In [ ]:
#load manual labels
all_human_labels  = pd.read_csv(mansc_path)
all_human_labels.columns = all_human_labels.iloc[0].str.strip("'")
all_human_labels = all_human_labels.iloc[1:-1,1:]
all_human_labels = all_human_labels.astype(int)

In [6]:
results = []
bad = 0

for path in tqdm(paths):
    #numerical IDs do repeat, so a partial string value is included
    id = path[-16:-4]
    rat_human_labeled = all_human_labels[[col for col in all_human_labels.columns if id in col]]
    print(f'id is :{id}')
    print(rat_human_labeled.columns)
    data = mne.io.read_raw_edf(path)
    #get relevant info
    raw_data = data.get_data()
    channels = data.ch_names
    #discard data with shitty electrodes or unclear placement
    if 'frontal' not in channels and 'EEG 1 frontal' not in channels and 'FRONTAL' not in channels:
        print('shitty labeling')
        bad += 1
        continue    
    
    #convert to dataframe
    data_df = pd.DataFrame(raw_data).T
    data_df.columns = channels
    
    #now try to apply my method to this data
    #only taking EMG and frontal for analysis
    if 'EEG 1 frontal' in channels:
        frontal = 'EEG 1 frontal' 
    elif 'frontal' in channels:
        frontal = 'frontal' 
    elif 'FRONTAL' in channels:
        frontal = 'FRONTAL' 
    else:
        print('no frontal in channels')
        continue
    
    if 'EMG' in channels:
        emg = 'EMG'
    elif 'EMG real' in channels:
        emg = 'EMG real'
    else:
        print('no emg in channels')
        continue
    
    #filtering
    filtered_data=pd.DataFrame()
    filtered_data['ecog']=filter_channel(data_df[frontal], fstart=0.5, fstop=45, sr=SAMPLE_RATE, center=True, notch_50=False)
    filtered_data['emg']=filter_channel(data_df[emg], fstart=5, fstop=100, sr=SAMPLE_RATE, center=True, notch_50=True)
    filtered_data.index=data_df.index
    #get spectral components
    BANDWIDTHS = {
        'delta': (0.5, 4),
        'theta': (4, 8),
        'alpha': (8, 13),
        'beta': (13, 30),
        'gamma': (30, 45)
    }

    #create filters
    bandwidth_filters = {
        name: signal.butter(N=20, Wn=frange, btype='bp', output='sos',  fs=SAMPLE_RATE)
        for name, frange in BANDWIDTHS.items()
    }
    #create output df
    freq_comps=pd.DataFrame()
    freq_comps.index=filtered_data.index
    #split to freq comps
    for i, (b, filter) in enumerate(bandwidth_filters.items()):
        freq_comps[str(b)] = signal.sosfiltfilt(filter, filtered_data['ecog'])
    #add emg
    freq_comps['emg']=filtered_data['emg']

    #convert to power - result is 4 s intervals
    pows = []
    index = []
    for col in freq_comps.columns:
        pow,ind,var = signal_to_power(freq_comps[col], window=1000)
        pows.append(pow)
        index.append(ind)
    freq_powers = pd.DataFrame(pows).T
    freq_powers.columns = freq_comps.columns
    freq_powers.index = index[0]
    print('FREQ POWERS OBTAINED')
    #get power ratios
    #convert to relative power - take each row and divide it by the sum of that row
    #ignore EMG since it's not cortical and has different values - is basically always max if included
    RELATIVE = True
    if RELATIVE:
        for ind in freq_powers.index:
            freq_powers.loc[ind,freq_powers.columns[:-1]]=freq_powers.loc[ind,freq_powers.columns[:-1]]/freq_powers.loc[ind,freq_powers.columns[:-1]].sum()

    #also standardize cols
    scaler = StandardScaler()
    freq_powers[freq_powers.columns]=scaler.fit_transform(freq_powers[freq_powers.columns])

    #apply a rolling median filter 60 points
    freq_powers = freq_powers.rolling(60).median()#.fillna(method='bfill',inplace=True)
    freq_powers = freq_powers.bfill()

    #code for training a new clusterer
    MODE = 'train'
    if MODE == 'train':
        #try clustering 
        X = freq_powers.values
        #select subset to save memory
        idx = np.random.randint(len(X), size=50000)
        subset=X[idx]

        clusterer = AgglomerativeClustering(n_clusters=50, linkage='ward')
        cluster_labels = clusterer.fit_predict(subset)

        # inductive learning model to replicate agglomerative clustering on new data - way more efficient
        classifier = RandomForestClassifier(random_state=42)
        inductive_learner = InductiveClusterer(clusterer, classifier).fit(X)

        #predictions from inductive clusterer
        freq_powers['cluster']=inductive_learner.predict(X)
        freq_powers['cluster']=freq_powers['cluster'].astype('category')
    print('CLUSTERER TRAINED')
    #now sort cls
    gr=freq_powers[['delta','theta','alpha','beta','gamma','emg','cluster']].groupby('cluster', observed = False).mean()
    gr['cluster']=freq_powers[['delta','theta','alpha','beta','gamma','emg','cluster']].groupby('cluster', observed = False).mean().index
    gr['sleep']='unknown'
    #run a cluster sorting function
    w_cls, nrem_cls, rem_cls = sort_cls_by_criteria(gr)
    #add back to data
    #add sleep states to data
    freq_powers['sleep']='unknown'
    freq_powers.loc[freq_powers['cluster'].isin(w_cls), 'sleep'] = 'W'
    freq_powers.loc[freq_powers['cluster'].isin(nrem_cls), 'sleep'] = 'N'
    freq_powers.loc[freq_powers['cluster'].isin(rem_cls), 'sleep'] = 'R'
    freq_powers = freq_powers.ffill()
    freq_powers

    #remove REM detections if they follow W
    for i in range(0, len(freq_powers['sleep'])):
        if (i>=5) and (freq_powers['sleep'].iloc[i]=='R'):
            if 'W' in freq_powers['sleep'].iloc[i-5:i].tolist():
                freq_powers['sleep'].iloc[i]='W'

    freq_powers = freq_powers.reset_index(drop = True)
    freq_powers['human_labeled'] = rat_human_labeled[0:len(freq_powers)].to_numpy()
    freq_powers['human_labeled'] = freq_powers['human_labeled'].replace({0:'W',1:'N',2:'R'})
    freq_powers['id'] = id
    results.append(freq_powers)
    print(f'RAT {id} DONE')
print(f'num recs analyzed: {len(paths)-shitty}')

In [7]:
pickle.dump(pd.concat(results), open('./res_temp/human_comparison.pkl','wb'))

In [8]:
results = pickle.load(open('./res_temp/human_comparison.pkl','rb'))

In [10]:
#redo sleep sorting here if want to - needs to be done by rat
#messing with params doesn't really help
for rat in results['id'].unique():
    results.loc[results['id']==rat, 'sleep'] = results.loc[results['id']==rat, 'sleep'].replace({0:'W',1:'N',2:'R'})
    gr=results.loc[results['id']==rat,['delta','theta','alpha','beta','gamma','emg','cluster']].groupby('cluster', observed = False).mean()
    gr['cluster']=results.loc[results['id']==rat,['delta','theta','alpha','beta','gamma','emg','cluster']].groupby('cluster', observed = False).mean().index
    gr['sleep']='unknown'
    #run a cluster sorting function
    w_cls, nrem_cls, rem_cls = sort_cls_by_criteria_alt(gr)
    #add back to data
    #add sleep states to data
    results.loc[results['id']==rat,'sleep']='unknown'
    results.loc[(results['id']==rat)&(results['cluster'].isin(w_cls)), 'sleep'] = 'W'
    results.loc[(results['id']==rat)&(results['cluster'].isin(nrem_cls)), 'sleep'] = 'N'
    results.loc[(results['id']==rat)&(results['cluster'].isin(rem_cls)), 'sleep'] = 'R'
results = results.ffill()
results

In [11]:
#non-smoothened
(results['sleep']==results['human_labeled']).sum()/len(results)

0.7479304597435067

In [ ]:
#smoothing
for rat in results['id'].unique():
    # results.loc[results['id']==rat, 'human_labeled'] = results.loc[results['id']==rat, 'human_labeled'].replace({0:'W',1:'N',2:'R'})
    results.loc[results['id']==rat, 'sleep'] = results.loc[results['id']==rat, 'sleep'].replace({0:'W',1:'N',2:'R'})
    
    integration_s = 5
    # results.loc[results['id']==rat, 'human_labeled']=pd.Series(integrate_by_mode(results.loc[results['id']==rat, 'human_labeled'], integration_s=integration_s))
    results.loc[results['id']==rat, 'sleep'] = pd.Series(integrate_by_mode(results.loc[results['id']==rat, 'sleep'], integration_s=integration_s))

In [ ]:
#smoothened
results = results.dropna()
(results['sleep']==results['human_labeled']).sum()/len(results)

In [14]:
#conv sleep states to category
from pandas.api.types import CategoricalDtype
cat_dtype = CategoricalDtype(categories = ['W','N','R'], ordered = True)
results['sleep'] = results['sleep'].astype(cat_dtype)
results['human_labeled'] = results['human_labeled'].astype(cat_dtype)

In [ ]:
#by rat
for rat in results['id'].unique():
    print((results.loc[results['id']==rat, 'sleep'] == results.loc[results['id']==rat, 'human_labeled']).sum()/len(results.loc[results['id']==rat]))

In [ ]:
#full evaluation
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
y_true = results['human_labeled'].dropna()
y_pred = results['sleep'].dropna()
# Example true labels and predicted labels for a multiclass classification problem

# Calculate precision, recall, and F1 score for each class
precision_per_class = precision_score(y_true, y_pred, average=None, labels = ['W','N','R'])
recall_per_class = recall_score(y_true, y_pred, average=None)
f1_per_class = f1_score(y_true, y_pred, average=None)

accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy}\n')

# # Print class-wise metrics
print("Class-wise Metrics:")
for i in range(len(precision_per_class)):
    print("Class {}: Precision={}, Recall={}, F1 Score={}".format(i, precision_per_class[i], recall_per_class[i], f1_per_class[i]))

# # Calculate and print confusion matrix
# plt.set_cmap('bwr')
conf_matrix = confusion_matrix(y_true, y_pred, labels = ['W','N','R'], normalize = 'true')
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix,display_labels=['W','N','R'])
disp.plot()
disp.ax_.set(xlabel='Automatically predicted label', ylabel='Human label')#('Automatically predicted label')#,ylabel = 'Human label')
disp.ax_.get_images()[0].set_clim(0, 1)
disp.ax_.get_images()[0].set_cmap('bwr')
plt.savefig('plots/conf_matrix.png', dpi = 300, transparent = True)

In [ ]:
#now get descriptions
results.groupby('human_labeled').median(numeric_only = True).apply(minmax_scale)

In [ ]:
results.groupby('sleep').median(numeric_only = True).apply(minmax_scale)

In [ ]:
#save to excel
results.groupby('human_labeled').median(numeric_only = True).apply(minmax_scale).to_excel('results/human_labeled_state_params.xlsx')
results.groupby('sleep').median(numeric_only = True).apply(minmax_scale).to_excel('results/clusterer_state_params.xlsx')

In [ ]:
#count
results.groupby('human_labeled').count()/len(results)

In [ ]:
#count
results.groupby('sleep').count()/len(results)

In [ ]:
#save to excel
(results.groupby('human_labeled').count()/len(results)).to_excel('results/human_labeled_counts.xlsx')
(results.groupby('sleep').count()/len(results)).to_excel('results/clusterer_counts.xlsx')

In [ ]:
#get state durations and counts
results

In [ ]:
durations_human = []
for id in results['id'].unique():
    duration_analysis = results.loc[results['id']==id].drop(['delta','theta','alpha','beta','gamma', 'emg'], axis = 1)
    duration_analysis['sleep_numeric'] = duration_analysis['human_labeled'].replace({'W':0,'N':1, 'R':2}).astype(int)
    duration_analysis['diff'] = duration_analysis['sleep_numeric'].diff()
    state_switches = duration_analysis.loc[(duration_analysis['diff']!=0)]
    state_switches['duration'] = 0
    state_switches['duration'].iloc[0:-1] = state_switches.index.to_series().diff().dropna().to_numpy()#already in s from previous analysis
    state_switches['id'] = id
    durations_human.append(state_switches)
durations_human = pd.concat(durations_human)

In [ ]:
durations_human.groupby('human_labeled').mean(numeric_only=True)

In [ ]:
durations_human.groupby('human_labeled').count()/len(durations_human['id'].unique())

In [ ]:
durations_auto = []
for id in results['id'].unique():
    duration_analysis = results.loc[results['id']==id].drop(['delta','theta','alpha','beta','gamma', 'emg'], axis = 1)
    duration_analysis['sleep_numeric'] = duration_analysis['sleep'].replace({'W':0,'N':1, 'R':2}).astype(int)
    duration_analysis['diff'] = duration_analysis['sleep_numeric'].diff()
    state_switches = duration_analysis.loc[(duration_analysis['diff']!=0)]
    state_switches['duration'] = 0
    state_switches['duration'].iloc[0:-1] = state_switches.index.to_series().diff().dropna().to_numpy()#already in s from previous analysis
    state_switches['id'] = id
    durations_auto.append(state_switches)
durations_auto = pd.concat(durations_auto)

In [ ]:
durations_auto.groupby('human_labeled').mean(numeric_only=True)

In [ ]:
durations_auto.groupby('human_labeled').count()/len(durations_human['id'].unique())

In [ ]:
#correct
correct = results.loc[results['sleep']==results['human_labeled']]
#sleep params where my classification was incorrect
correct.groupby('human_labeled').median(numeric_only=True).apply(minmax_scale)

In [ ]:
#figure out where my method makes mistakes
mistakes = results.loc[results['sleep']!=results['human_labeled']]
#sleep params where my classification was incorrect
mistakes.groupby('human_labeled').median(numeric_only=True).apply(minmax_scale)

In [33]:
#plot - single rat
rats = results['id'].unique()
rat = rats[0]
single_rat = results[results['id']==rat]

In [ ]:
fig, axs = plt.subplots(2,1,figsize=(10,6))
time = np.arange(0,len(single_rat))/(15*60)
axs[0].plot(time, single_rat['sleep'])
axs[0].set(title = 'Rule-based classifier', xlabel = 'Time, hours from start', ylabel = 'Sleep state', xlim = (-0.5,24.5))
ymin, ymax = axs[0].get_ylim()
axs[0].vlines(time[int(np.floor(len(time)/2))], ymin, ymax, color = 'k', linestyles = '--')
axs[0].vlines(time[0], ymin, ymax, color = 'r', linestyles = '--')
axs[0].locator_params(axis = 'x', nbins = 24)

axs[1].plot(time, single_rat['human_labeled'])
axs[1].set(title = 'Human labels', xlabel = 'Time, hours from start', ylabel = 'Sleep state', xlim = (-0.5,24.5))
ymin, ymax = axs[1].get_ylim()
axs[1].vlines(time[int(np.floor(len(time)/2))], ymin, ymax, color = 'k', linestyles = '--')
axs[1].vlines(time[0], ymin, ymax, color = 'r', linestyles = '--')
axs[1].locator_params(axis = 'x', nbins = 24)

plt.tight_layout()
plt.savefig('./plots/classifiervshuman.png')
plt.show()

In [35]:
bool_cat = CategoricalDtype(categories = ['Yes','No'], ordered=True)

In [ ]:
#now check where mistakes happens
fig, axs = plt.subplots(3,1,figsize=(10,8))
time = np.arange(0,len(single_rat))/(15*60)
axs[0].plot(time, single_rat['sleep'])
axs[0].set(title = 'Rule-based classifier', xlabel = 'Time, hours from start', ylabel = 'Sleep state', xlim = (-0.5,24.5))
ymin, ymax = axs[0].get_ylim()
axs[0].vlines(time[int(np.floor(len(time)/2))], ymin, ymax, color = 'k', linestyles = '--')
axs[0].vlines(time[0], ymin, ymax, color = 'r', linestyles = '--')
axs[0].locator_params(axis = 'x', nbins = 24)

axs[1].plot(time, single_rat['human_labeled'])
axs[1].set(title = 'Human labels', xlabel = 'Time, hours from start', ylabel = 'Sleep state', xlim = (-0.5,24.5))
ymin, ymax = axs[1].get_ylim()
axs[1].vlines(time[int(np.floor(len(time)/2))], ymin, ymax, color = 'k', linestyles = '--')
axs[1].vlines(time[0], ymin, ymax, color = 'r', linestyles = '--')
axs[1].locator_params(axis = 'x', nbins = 24)

axs[2].plot(time, (single_rat['human_labeled']==single_rat['sleep']).replace({True:'Yes',False:'No'}).astype(bool_cat))
axs[2].set(title = 'Human labels', xlabel = 'Time, hours from start', ylabel = 'Do labels match?', xlim = (-0.5,24.5))
ymin, ymax = axs[2].get_ylim()
axs[2].vlines(time[int(np.floor(len(time)/2))], ymin, ymax, color = 'k', linestyles = '--')
axs[2].vlines(time[0], ymin, ymax, color = 'r', linestyles = '--')
axs[2].locator_params(axis = 'x', nbins = 24)

plt.tight_layout()
plt.savefig('./plots/classifiervshumanwithdiff.png')
plt.show()

In [ ]:
#now check where mistakes happens
single_hour = single_rat.iloc[2000:3000]
fig, axs = plt.subplots(3,1,figsize=(10,8))
time = np.arange(0,len(single_hour))/(15)
axs[0].plot(time, single_hour['sleep'])
axs[0].set(title = 'Rule-based classifier', xlabel = 'Time, minutes', ylabel = 'Sleep state')#, xlim = (-0.5,24.5))
axs[0].locator_params(axis = 'x', nbins = 24)

axs[1].plot(time, single_hour['human_labeled'])
axs[1].set(title = 'Human labels', xlabel = 'Time, minutes', ylabel = 'Sleep state')#, xlim = (-0.5,24.5))
axs[1].locator_params(axis = 'x', nbins = 24)

axs[2].plot(time, (single_hour['human_labeled']==single_hour['sleep']).replace({True:'Yes',False:'No'}).astype(bool_cat))
axs[2].set(title = 'Human labels', xlabel = 'Time, minutes', ylabel = 'Do labels match?')#, xlim = (-0.5,24.5))
axs[2].locator_params(axis = 'x', nbins = 24)

plt.tight_layout()
plt.savefig('./plots/classifiervshumanwithdiffsinglehour.png')
plt.show()